# Example of running sar-pre-processing package

## 1. Requirements

- Installation of ESA's SNAP Toolbox (https://step.esa.int/main/download/snap-download/)
- Sentinel-1 SLC data
    - location where files are stored (locally)<br>
    **or**  
    - use 2. to download data from the internet
        - Free account for Copernicus Sentinel Data Hub needs to be created (https://scihub.copernicus.eu/dhus/#/self-registration)
        - python package "sentinelsat version 0.13" will be installed
        - may require some free disk space

## 2. Download sample data from Sentinel Data Hub

In [1]:
# Install python package sentinelsat
! pip install sentinelsat==0.13 --user

### <span style="color:red">Create Account (https://scihub.copernicus.eu/dhus/#/self-registration) and change user and password below</span>.

In [2]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
user = 'username'
password = 'password'

# initialize settings
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/apihub/')

In [3]:
# search by polygon (MNI test site coordinates), time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('coordinates_mni.geojson'))
products = api.query(footprint,
                     date=('20200101', '20200120'),
                     platformname='Sentinel-1',
                     producttype='SLC')
print('Following products will be downloaded')
print(api.to_dataframe(products).title.values)

print('These {} product need {} Gb of disk space'.format(len(products), api.get_products_size(products)))

Following products will be downloaded
['S1A_IW_SLC__1SDV_20200119T052619_20200119T052647_030865_038AB8_A3E8'
 'S1B_IW_SLC__1SDV_20200116T165849_20200116T165916_019845_025868_C2F7'
 'S1B_IW_SLC__1SDV_20200116T165823_20200116T165851_019845_025868_7734'
 'S1A_IW_SLC__1SDV_20200115T170724_20200115T170751_030814_0388F3_901F'
 'S1A_IW_SLC__1SDV_20200114T051810_20200114T051838_030792_038823_B323'
 'S1B_IW_SLC__1SDV_20200113T052538_20200113T052605_019794_0256CD_CFF9'
 'S1A_IW_SLC__1SDV_20200110T165932_20200110T170000_030741_03864C_E827'
 'S1A_IW_SLC__1SDV_20200110T165907_20200110T165934_030741_03864C_88B8'
 'S1B_IW_SLC__1SDV_20200109T170642_20200109T170709_019743_025539_BDD7'
 'S1B_IW_SLC__1SDV_20200108T051746_20200108T051813_019721_025492_422D'
 'S1A_IW_SLC__1SDV_20200107T052619_20200107T052647_030690_038490_1F39'
 'S1B_IW_SLC__1SDV_20200104T165824_20200104T165852_019670_0252D8_3D51'
 'S1B_IW_SLC__1SDV_20200104T165849_20200104T165916_019670_0252D8_0278'
 'S1A_IW_SLC__1SDV_20200103T170725_2020

**Start download process (<span style="color:red">Attention: might take a while and data will requries some free disk space</span>)**

In [4]:
import os

# download all results from the search
# files will be downloaded in subolder "test_files" of current directory
api.download_all(products,os.getcwd()+'/test_files')

(OrderedDict([('9bbb44fd-687f-46f6-8ab6-a03a3978dfb0',
               {'id': '9bbb44fd-687f-46f6-8ab6-a03a3978dfb0',
                'title': 'S1A_IW_SLC__1SDV_20200119T052619_20200119T052647_030865_038AB8_A3E8',
                'size': 4890155753,
                'md5': '8A28A2076E5721A001F9D6B76FB7B410',
                'date': datetime.datetime(2020, 1, 19, 5, 26, 19, 467000),
                'footprint': 'POLYGON((12.721289 47.890324,9.288409 48.291393,9.678197 49.963123,13.230704 49.560402,12.721289 47.890324))',
                'url': "https://scihub.copernicus.eu/apihub/odata/v1/Products('9bbb44fd-687f-46f6-8ab6-a03a3978dfb0')/$value",
                'Online': True,
                'Creation Date': datetime.datetime(2020, 1, 19, 8, 44, 0, 159000),
                'Ingestion Date': datetime.datetime(2020, 1, 19, 8, 43, 4, 426000),
                'path': '/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/notebooks/test_files/S1A_IW_SLC__1SDV_20200119T052619_20200119T

## 2. Use sar-pre-processing package to process Sentinel-1 SLC data

Set paths for
- input_folder (location with stored Sentinel-1 SLC data)
- output_folder (location where processed data will be stored)
- gpt_loction (location of ESA's SNAP toolbox installation, usually something like 'home/snap/bin/gpt' 

In [2]:
import os
input_folder = os.getcwd()+'/test_files'
output_folder = os.getcwd()+'/test_files'
gpt_location = '/home/tweiss/snap/bin/gpt'

Create config file with information about input, output and gpt loction

In [6]:
import yaml

with open('sample_config_file.yaml') as stream:
   data = yaml.safe_load(stream)

data['input_folder'] = input_folder
data['output_folder'] = output_folder
data['gpt'] = gpt_location

with open('test_config_file.yaml', 'wb') as stream:
   yaml.safe_dump(data, stream, default_flow_style=False, 
                  explicit_start=True, allow_unicode=True, encoding='utf-8')

Start pre-processing steps

In [9]:
from sar_pre_processing.sar_pre_processor import *
import warnings
warnings.filterwarnings("ignore")

processing = SARPreProcessor(config='test_config_file.yaml')
processing.create_processing_file_list()
print('start step 1')
processing.pre_process_step1()
print('start step 2')
processing.pre_process_step2()
print('start step 3')
processing.pre_process_step3()
print('start solve projection problem')
processing.solve_projection_problem()
print('start add netcdf information')
processing.add_netcdf_information()
print('start create netcdf stack')
processing.create_netcdf_stack()

INFO:root:Found files within input folder: 16
INFO:root:Number of found files for year 2020: 16
INFO:root:Number of found files containing area of interest: 16
INFO:root:Number of found files that were double processed: 0.0
INFO:root:Number of found files with border issues: 6
INFO:root:normalisation angle not specified, default value of 35 is used for processing
INFO:ComponentProgress:0
INFO:ComponentProgress:0
INFO:root:Process S1A_IW_SLC__1SDV_20200102T051811_20200102T051839_030617_038202_B1AD.zip with SNAP.


start step 1


INFO:root:0
INFO:ComponentProgress:6
INFO:ComponentProgress:6
INFO:root:Process S1A_IW_SLC__1SDV_20200103T170725_20200103T170752_030639_0382D5_7345.zip with SNAP.
INFO:root:0
INFO:ComponentProgress:12
INFO:ComponentProgress:12
INFO:root:Process S1A_IW_SLC__1SDV_20200107T052619_20200107T052647_030690_038490_1F39.zip with SNAP.
INFO:root:0
INFO:ComponentProgress:18
INFO:ComponentProgress:18
INFO:root:Process S1A_IW_SLC__1SDV_20200114T051810_20200114T051838_030792_038823_B323.zip with SNAP.
INFO:root:0
INFO:ComponentProgress:25
INFO:ComponentProgress:25
INFO:root:Process S1A_IW_SLC__1SDV_20200115T170724_20200115T170751_030814_0388F3_901F.zip with SNAP.
INFO:root:0
INFO:ComponentProgress:31
INFO:ComponentProgress:31
INFO:root:Process S1A_IW_SLC__1SDV_20200119T052619_20200119T052647_030865_038AB8_A3E8.zip with SNAP.
INFO:root:0
INFO:ComponentProgress:37
INFO:ComponentProgress:37
INFO:root:Process S1B_IW_SLC__1SDV_20200101T052539_20200101T052606_019619_025146_5770.zip with SNAP.
INFO:root:0


start step 2


INFO:root:0
INFO:root:2020-08-03 12:52:32.730690
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:Scene 2 of 13
INFO:root:Process S1A_IW_SLC__1SDV_20200103T170725_20200103T170752_030639_0382D5_7345_GC_RC_No_Su.dim with SNAP.
INFO:root:0
INFO:root:2020-08-03 12:53:02.544946
INFO:ComponentProgress:15
INFO:ComponentProgress:15
INFO:root:Scene 3 of 13
INFO:root:Process S1A_IW_SLC__1SDV_20200107T052619_20200107T052647_030690_038490_1F39_GC_RC_No_Su.dim with SNAP.
INFO:root:0
INFO:root:2020-08-03 12:53:31.357838
INFO:ComponentProgress:23
INFO:ComponentProgress:23
INFO:root:Scene 4 of 13
INFO:root:Process S1A_IW_SLC__1SDV_20200110T165907_20200110T165934_030741_03864C_88B8_GC_RC_No_Su.dim with SNAP.
INFO:root:0
INFO:root:2020-08-03 12:53:56.158006
INFO:ComponentProgress:30
INFO:ComponentProgress:30
INFO:root:Scene 5 of 13
INFO:root:Process S1A_IW_SLC__1SDV_20200114T051810_20200114T051838_030792_038823_B323_GC_RC_No_Su.dim with SNAP.
INFO:root:0
INFO:root:2020-08-03 12:54:19.409459
I

start step 3


INFO:root:0
INFO:root:2020-08-03 12:58:57.394939
INFO:ComponentProgress:7
INFO:ComponentProgress:7
INFO:root:0
INFO:root:2020-08-03 12:59:35.080857
INFO:ComponentProgress:15
INFO:ComponentProgress:15
INFO:root:0
INFO:root:2020-08-03 13:00:13.526594
INFO:ComponentProgress:23
INFO:ComponentProgress:23
INFO:root:0
INFO:root:2020-08-03 13:00:52.142897
INFO:ComponentProgress:30
INFO:ComponentProgress:30
INFO:root:0
INFO:root:2020-08-03 13:01:30.956110
INFO:ComponentProgress:38
INFO:ComponentProgress:38
INFO:root:0
INFO:root:2020-08-03 13:02:10.434079
INFO:ComponentProgress:46
INFO:ComponentProgress:46
INFO:root:0
INFO:root:2020-08-03 13:02:49.725604
INFO:ComponentProgress:53
INFO:ComponentProgress:53
INFO:root:0
INFO:root:2020-08-03 13:03:28.248374
INFO:ComponentProgress:61
INFO:ComponentProgress:61
INFO:root:0
INFO:root:2020-08-03 13:04:06.162595
INFO:ComponentProgress:69
INFO:ComponentProgress:69
INFO:root:0
INFO:root:2020-08-03 13:04:45.088142
INFO:ComponentProgress:76
INFO:ComponentProg

start solve projection problem
start add netcdf information
start create netcdf stack
Number of scenes found for processing: 13

Scene 1 of 13
/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/notebooks/test_files/step3/S1B_IW_SLC__1SDV_20200101T052539_20200101T052606_019619_025146_5770_GC_RC_No_Su_Co_speckle.nc

Scene 2 of 13
/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/notebooks/test_files/step3/S1A_IW_SLC__1SDV_20200102T051811_20200102T051839_030617_038202_B1AD_GC_RC_No_Su_Co_speckle.nc

Scene 3 of 13
/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/notebooks/test_files/step3/S1A_IW_SLC__1SDV_20200103T170725_20200103T170752_030639_0382D5_7345_GC_RC_No_Su_Co_speckle.nc

Scene 4 of 13
/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/notebooks/test_files/step3/S1B_IW_SLC__1SDV_20200104T165824_20200104T165852_019670_0252D8_3D51_GC_RC_No_Su_Co_speckle.nc

Scene 5 of 13
/media/tweiss/Work/GIT/GitHub/multiply-org/sar-pre-processing/not

## 3. View processed data

Load netcdf file with processed data

In [3]:
! pip install netCDF4 --user
from netCDF4 import Dataset
import numpy as np

my_example_nc_file = output_folder + '/test_files.nc'
data = Dataset(my_example_nc_file, mode='r')

View information about dataset

In [4]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lat(2404), lon(2404), time(13)
    variables(dimensions): float32 time(time), float32 orbitdirection(time), float32 relorbit(time), float32 satellite(time), float32 lat(lat), float32 lon(lon), float32 localIncidenceAngle(time,lat,lon), float32 sigma0_vv_multi(time,lat,lon), float32 sigma0_vh_multi(time,lat,lon), float32 sigma0_vv_norm_multi(time,lat,lon), float32 sigma0_vh_norm_multi(time,lat,lon)
    groups: 

Read data from netcdf file

In [5]:
data.variables['orbitdirection'][:]
data.variables['time'][:]
lons = data.variables['lon'][:]
lats = data.variables['lat'][:]
vv = data.variables['sigma0_vv_multi'][:]

vv_units = data.variables['sigma0_vv_multi'].units

Close netcdf file

In [6]:
data.close()

Plot vv polorized data

In [7]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    # Problem: border pixel might be zero or negative
    # pixel eqal or smaller than zero are set to nan
    array = np.copy(vv[x])
    array[array <= 0] = np.nan
    # plot backscatter data in dB scale
    plt.imshow(10*np.log10(array))
    cbar = plt.colorbar()
    cbar.set_label('dB')
    plt.clim(-25, 0)

interactive_plot = interactive(f, x=(0,len(vv)-1))
interactive_plot

interactive(children=(IntSlider(value=6, description='x', max=12), Output()), _dom_classes=('widget-interact',…